In [1]:
import pandas as pd
import numpy as np
from numpy import nan as NA
from pandas import DataFrame, Series
import re # regular expressions
#adding graphics
import matplotlib.pyplot as plt
%pylab inline
from pylab import rcParams
from collections import defaultdict
from collections import OrderedDict as od
rcParams['figure.figsize'] = 18, 6
print 'test'

Populating the interactive namespace from numpy and matplotlib
test


In [2]:
#all_df = pd.read_table('balances.csv', sep=',',index_col='Ref_Date') 
all_lic = pd.read_table('business.licences.csv', sep=',') 

#fix the dates
def addyear(x):
    ystr = x[6]+x[7]
    if (ystr>'18') :
        ystr = '19'+ystr
    else :
        ystr = '20'+ystr
    return ystr

def getpostal(postal) :
    return postal[:3]+ postal[4:]
def getarea(po) :
    area = po[:3]
    return area
# get rid of nulls in postal codes
all_lic= all_lic[pd.notnull(all_lic['Licence Address Line 3'])]
all_lic['year']=all_lic["Issued"].map(addyear)
all_lic['postal']=all_lic['Licence Address Line 3'].map(getpostal)
all_lic['area']=all_lic['postal'].map(getarea)
#all_lic['lat']=all_lic["postal"].map(lat)
#all_lic['lng']=all_lic["postal"].map(lng)
print all_lic['postal']

0         M9L1N9
1         M1W2Z4
2         M1V2M1
3         M9W2Z2
4         M9W5E3
5         M4B1P2
6         M1L1M5
7         M1N1R3
8         M1C1L5
9         M5V3B1
10        M5V3B1
11        M1V4B5
12        M1K1E7
13        M4B1P2
14        M5A2R3
15        M5A2R3
16        M8Z1A1
17        M2M3Y2
18        M2M3Y2
19        M1H1A5
20        M1H1A5
21        M8Y1A9
22        M4B2V2
23        M9W5E3
24        M4G3V3
25        M8Y1A9
26        M1M1R4
27        M4G3V3
28        M4G3V3
29        M3K1E2
           ...  
144461       M4J
144462       M6P
144463       M8Z
144464       M8Z
144465       M6K
144466       M8Z
144467       M4H
144468       M9L
144469       M3J
144470       M4K
144471       L4C
144472       M5H
144473       M8Z
144474       L5R
144475       M8Z
144476       M8Z
144477       M8Z
144478       M8Z
144479       M4L
144480       M6H
144481       M1C
144482       M3H
144483       L6B
144484       M4K
144485       M5R
144486       L9P
144487       M1G
144488       L

/Users/ahmed/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (6,7,8,13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
cat = all_lic['Category']
d = defaultdict(int)

for c in cat :   
    d[c]+=1

df = DataFrame(d.items(),columns=['category','count'])
indf = df.set_index(['category'])
print indf.shape

cat500up = indf[indf['count']>=500]
cat100_500 = indf[indf['count']>=100]
cat100_500 = cat100_500[cat100_500['count']<500]
cat100down = indf[indf['count']<100]
#print cat500up.shape
#print cat100_500.shape
#print cat100down.shape
#print 
#indf['count'].plot()
#for attrib in d:
#        print attrib
#highcat = cat500up[cat500up['count']>5000]
#highcat.plot

(82, 1)


In [4]:
#Quickly fix the dates by adding 19 and 20


def buildCategory(cat):
    cat_target= all_lic [all_lic['Category']==cat]
   
    #select the fields we want
    target=cat_target[['Operating Name','Issued','year','postal','area']]
    dest='categories/'+cat + '.csv'
    target.to_csv(dest)
    return target

In [5]:
def limit_year(dataset,start=None,end=None,year=None) :

    target = dataset
    if (start is not None):
        print " filtering for start year " + start
        target= target [target['year']>= start]
    if (end is not None) :
        print " filtering for end year " + end
        target = target[target['year']<= end]
    if (year is not None) :
        print " filtering for  year " + year
        target = target[target['year']== year]
    return target
        

In [45]:
rest2000 = limit_year(buildCategory(cat='EATING ESTABLISHMENT'),year='2004')
print rest2000

 filtering for  year 2004
                                       Operating Name    Issued  year  postal  \
64596                          CHINA FEAST RESTAURANT  01-01-04  2004  M4G2Z8   
64669                       CHIN CHYN CHINESE CUISINE  14-01-04  2004  M1P4Y4   
64754                  AL-MADINA HALAL PIZZA & BAKERY  23-01-04  2004  M1P2S2   
64790                                   TEA POT CHATT  01-01-04  2004  M4X1P8   
64791                EL CAMINO BAR RESTAURANT MEXICAN  19-02-04  2004  M6P1Y7   
64805                KHAWAJA GHAREEB NAWAZ RESTAURENT  07-01-04  2004  M4L3B4   
64811                    FLOWER DRUM RESTAURANT & BAR  16-02-04  2004  M8V1C4   
64822                  SANKKAMAM TAKEOUT AND CATERING  12-01-04  2004  M1G3S3   
64827                             NIAGARA STREET CAFE  02-03-04  2004  M5V1C9   
64851                  STREAMS OF BLESSING FISH SHACK  01-01-04  2004  M5T2M1   
64857  EL CAMINO REAL BAR AND RESTAURANT MEXICAN FOOD  20-02-04  2004  M6P1Y7   
64

In [46]:
# in this part we read the ontario csv to lookup coordinates
onp = pd.read_table('ontario.csv', sep=',') 
print onp.shape

def getpostalcoordinates (code):
    postal = onp[onp['postal']==code]
    return postal

def coord(x) :
    c = getpostalcoordinates(x)
    if c is None :
        return 0,0
    if c.empty:
        return 0,0
    return c.iloc[0]['lat'],c.iloc[0]['lng']

temp= coord('M6G1K7')
print temp

# here we build a list of all zip codes in a list
def getweighted(dataset) :
    target = dataset
    areadict = defaultdict(int)
    for c in dataset['area'] :   
        areadict[c]+=1
    f = lambda x : areadict[x]
    target['w'] = target['area'].map(f)
    target['lat'],target['lng']= zip(*target['postal'].map(coord))
    return target
testw = getweighted(rest2000)

(313072, 7)
(43.664636000000002, -79.414206000000007)


In [41]:
print testw

                                Operating Name    Issued  year  postal area  \
80795                                 GEORGE'S  06-02-14  2014  M6E1A1  M6E   
81015  ROCHA AND DIAS BAKERY AND CHURRASQUEIRA  06-02-14  2014  M6E2V8  M6E   
81043                           LICKEN CHICKEN  10-04-14  2014  M4H1P9  M4H   
81048                            DIM SUM HOUSE  13-02-14  2014  M1V5N1  M1V   
81169            HOT WINGS GRILL AND RIB HOUSE  03-01-14  2014  M5V2B6  M5V   
81208                              AMPM COFFEE  29-04-14  2014  M5J2T5  M5J   
81268                          PORTA NOVA CAFE  06-02-14  2014  M6H2C1  M6H   
81309                     GOOD TIMES BAD TIMES  08-04-14  2014  M6P3L4  M6P   
81310           UNITED COFFEE AND ESPRESSO INC  22-04-14  2014  M6G3T2  M6G   
81405                        NEW AKASH TAKEOUT  31-01-14  2014  M1W2L1  M1W   
81428                          GRINDERTOGO.COM  15-01-14  2014  M4M1Z6  M4M   
81435                     JETSUN'S JUICYBURGER  25-0

In [42]:
testw1=testw[testw['lat']==0]
print testw1

                   Operating Name    Issued  year  postal area   w  lat  lng
89466       FRESH START COFFEE CO  24-06-14  2014  M5E2K4  M5E  16    0    0
90049      CHIPOTLE MEXICAN GRILL  28-11-14  2014  M6N0A3  M6N  24    0    0
90086  LEFT BANK GRILL & WINE BAR  09-12-14  2014  M4M4A5  M4M  32    0    0


In [43]:
testw=testw[testw['lat']!=0]
testw= testw[['lat','lng','w']]
heatmapdata= testw.to_records(index=False)

In [44]:
import gmaps
print heatmapdata[:5]
m = gmaps.heatmap(heatmapdata, max_intensity=10, point_radius=5)
gmaps.display(m)

[(43.679671, -79.436361, 27) (43.685834, -79.437997, 27)
 (43.70422, -79.350663, 5) (43.818529, -79.297535, 43)
 (43.647689, -79.400639, 63)]
